# 0. 필수 라이브러리 로드

In [110]:
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver as wd
import time
import pandas as pd
import random
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 1. 에브리타임 강의평가 글 크롤링하는 함수 정의

In [111]:
def review_scraping(user_agent, path, my_id, my_password, lecture_name, prof_name, campus):
    url = 'https://everytime.kr' 
    options = wd.ChromeOptions()

    options.add_argument("no-sandbox")

    options.add_argument("--disable-dev-shm-usage")
    
    options.add_argument(f"user-agent={user_agent}")

    options.add_argument('window-size=1920,1080')
    #path는 ='r/Users/alsrl/CUAI/' 처럼 마지막에 /를 꼭 붙여줘야함
        
    driver = wd.Chrome(executable_path = path + 'chromedriver.exe', options = options)
    driver.maximize_window()
    try:
        #브라우저 열기
        driver.get(url)

        #에브리타임이 아니면 오류 발생하는걸 알려줘라!(driver.title로 페이지 제목 확인 가능)
        assert "에브리타임" in driver.title 
        driver.find_element("xpath", "/html/body/aside/div[1]/a[2]").click()
    
    
        #아이디 비밀번호 입력
        driver.find_element("xpath", '//*[@id="container"]/form/p[1]/input').send_keys(my_id)
        driver.find_element("xpath", '//*[@id="container"]/form/p[2]/input').send_keys(my_password)

        driver.implicitly_wait(time_to_wait=2) #각 클릭마다 쉬게 해줌

        #로그인 클릭
        driver.find_element("xpath", '//*[@id="container"]/form/p[3]/input').click()

        driver.implicitly_wait(time_to_wait=2) 
        
        #강의평가 클릭
        driver.find_element("xpath", '//*[@id="menu"]/li[3]/a').click()

        driver.implicitly_wait(time_to_wait=2) 
        
        #검색어를 입력할 요소 찾기
        search_box = driver.find_element("xpath", '//*[@id="container"]/form/input[1]')

        #검색어 입력 & 엔터 
        search_box.send_keys(lecture_name)
        search_box.send_keys(Keys.ENTER)

        driver.implicitly_wait(time_to_wait=2) #각 클릭마다 쉬게 해줌
        
        #검색 결과 개수를 저장함
        h3_tag = driver.find_elements(By.TAG_NAME, 'h3')
        
        #강의평가를 저장할 빈 리스트
        review_lst = []

        #개수만큼 반복함
        for i in range(1, len(h3_tag) + 1):
                
            html = driver.page_source 
            soup = BeautifulSoup(html, 'html.parser')
            
            #검색결과에서 첫번째를 클릭
            driver.find_element("xpath", f'//*[@id="container"]/div/a[{i}]/h3/p[1]').click()
            driver.implicitly_wait(time_to_wait=2)
            
            #교수님 성함 불러오기
            p = driver.find_element('xpath','//*[@id="container"]/div[2]/p[2]/span').text
            # 캠퍼스 이름 불러오기
            c = driver.find_element('xpath','//*[@id="container"]/div[2]/p[1]/span').text
            
            #검색한 정보와 맞는지 검증
            if not (p==prof_name and c==campus):
                driver.implicitly_wait(time_to_wait=2)
                #뒤로 가야지 나중에 문제 없이 작동
                driver.back()
                continue
            
            #강의평가가 있는지 확인
            check_empty = driver.find_elements(By.CLASS_NAME, 'empty')
            driver.implicitly_wait(time_to_wait=2)
            
            if len(check_empty) != 3:
                html = driver.page_source
            
            
                soup = BeautifulSoup(html, 'html.parser')
            
                title = soup.find('h2')
            
                text = soup.select("article")            
        
            for num in range(len(text)):
                if text[num].select_one('div.pay') == None:
                    review = text[num].select_one('p.text').text
                    review_lst.append(review)

            driver.implicitly_wait(time_to_wait=2)
            driver.back()
        
        driver.quit()
        
        return review_lst
        
    except:
        error = '에러 발생'
        return error

# 3. 정의한 함수를 이용하여 에브리타임 데이터 가져오기

In [116]:
data = review_scraping('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
                       '', '개인 아이디', '개인 비밀번호', 'ACT','교수님 성함','서울캠퍼스')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_3928\3993985574.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = wd.Chrome(executable_path = path + 'chromedriver.exe', options = options)


In [117]:
print(data[0])
print(data[-1])

기말 팀 프로젝트가 제일 중요해요....다른 거 점수 잘 받아봤자.... 팀원 운이 진짜 좋아야할 것 같네요
교수님 너무 좋아용 말씀도 친절하시고 최고최고


# 4. 가져온 데이터를 데이터 프레임으로 전환하기

In [120]:
df = pd.DataFrame(data)

In [123]:
df.columns = ['document']
df

,document
0,기말 팀 프로젝트가 제일 중요해요....다른 거 점수 잘 받아봤자.... 팀원 운이...
1,교수님이 친절하시고 학생에게 배려를 많이 줍니다! 그래도 팀원을 잘 만나야 한학기 ...
2,들어보면 왜 평점이 좋은 수업인지 알 수 있습니다 학생 배려를 정말 많이 해주십니다...
3,기말동영상 ㅈ되면 커버 불가능합니다 개인동영상은 대충하든 열심히하든 다 만점인 걸로...
4,개인과제는 다 만점 주셨어요. 그래서 팀 프로젝트 반영이 크니까 팀 프로젝트 잘 해...
...,...
124,학생들에 대한 배려가 넘치시고 격려도 많이 해주십니다
125,팀원잘만나고 팀장 잘만나야돼요팀은 랜덤이니 기도하시고 또 기도하세요성실하게하는모습만...
126,비대면 수업이었는데 교수님 좋으시고 학생들 배려 많이 해주셨어요. 피드백도 꼼꼼히 ...
127,교수님 수업에 열정도 있으시고 격려와 재미난학습으로 즐겁게 배웠습니다


# 5. 데이터 내보내기

In [126]:
df.to_excel('et_data.xlsx', index=False)